# CityBikes

In [19]:
# Import necessary modules

import requests
import pandas as pd
import plotly.express as px
import datetime
import os
from bs4 import BeautifulSoup
import json

#### Exploring the API structure for the city bikes
url = 'http://api.citybik.es/v2/networks'

In [20]:
# Exploring the API structure for the city bikes
city_bik_networks = requests.get('http://api.citybik.es/v2/networks')
city_bik_networks_list = city_bik_networks.json()
if city_bik_networks.status_code == 200:
    print("Succesful connection with API.")
    print('-------------------------------')
    data = city_bik_networks.json()
elif city_bik_networks.status_code == 404:
    print("Unable to reach URL.")
else:
    print("Unable to connect API or retrieve data.")

Succesful connection with API.
-------------------------------


In [21]:
#print(data)
print(type(data))
#print(city_bik_networks.json())

<class 'dict'>


#####  Answer:
        Data returned is a JSON data and the structure is a dictionary

##### Choose a city covered by the CityBikes API and retrieve all available bike stations in that city.
##### (Send a request to CityBikes for the city of your choice)

In [22]:
import requests
import pandas as pd

# Fetch the list of all networks
url = "http://api.citybik.es/v2/networks"
response = requests.get(url)
data = response.json()

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Prompt the user for the city name
    city_name = input("Enter the name of the city: ").lower()

    # Filter networks by city name
    filtered_networks = [network for network in data["networks"] if network["location"]["city"].lower() == city_name]

    if filtered_networks:
        chosen_network = filtered_networks[0]
        chosen_network_id = chosen_network["id"]

        # Fetch network data for the chosen city
        chosen_network_url = f"http://api.citybik.es/v2/networks/{chosen_network_id}"
        chosen_network_response = requests.get(chosen_network_url)
        chosen_network_data = chosen_network_response.json()

        # Extract bike stations information
        bike_stations = chosen_network_data.get("network", {}).get("stations", [])

        if bike_stations:
            # Create a DataFrame from the bike_stations list
            available_bike_stations = pd.DataFrame(bike_stations)

            # Display bike station information
            print(f"Bike Stations in {city_name.capitalize()}:\n")
            print(available_bike_stations)
        else:
            print(f"No bike stations found for {city_name.capitalize()}.")
    else:
        print(f"No networks found for {city_name.capitalize()}.")
else:
    print(f"Failed to retrieve network data. Status code: {response.status_code}")


Bike Stations in Lille:

     empty_slots                                              extra  \
0             18  {'address': 'Parvis Rotterdam', 'city': 'LILLE...   
1             22  {'address': 'ALLEE VICTOR BASCH', 'city': 'LA ...   
2              0  {'address': 'RUE CHAMBORD', 'city': 'VILLENEUV...   
3              9  {'address': '205-189 rue de l'Epeule', 'city':...   
4              4  {'address': '2 RUE MARCEL HÉNAUX', 'city': 'LI...   
..           ...                                                ...   
284            3  {'address': '15 PLACE PHILLIPPE LEBON', 'city'...   
285            0  {'address': '69 AVENUE DE VERDUN', 'city': 'MO...   
286            0  {'address': '56, RUE PAUL VAILLANT', 'city': '...   
287           14  {'address': 'RUE CORNEILLE', 'city': 'VILLENEU...   
288           10  {'address': 'Angle Chatellenie / allée chantec...   

     free_bikes                                id   latitude  longitude  \
0             2  3d91e5c7edcdf745a745ae95b3cf37

In [23]:
available_bike_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  289 non-null    int64  
 1   extra        289 non-null    object 
 2   free_bikes   289 non-null    int64  
 3   id           289 non-null    object 
 4   latitude     289 non-null    float64
 5   longitude    289 non-null    float64
 6   name         289 non-null    object 
 7   timestamp    289 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 18.2+ KB


##### Another option:

In [24]:
# Choose a city covered by the CityBikes API and retrieve all available bike stations in that city

import requests

'''
    This function below takes a city name as parameter, 
    and returns its list of stations' info, only if the city name 
    matches with exactly one city bike company.

    If there isn't exactly one city bike company for a given string, 
    print an error message and return None.
    '''


def get_city_data(city):    
    city_bike_networks = requests.get("http://api.citybik.es/v2/networks").json()

    list_of_dicts = []

    for city_bike_dict in city_bike_networks['networks']:
        new_city = city_bike_dict['location']['city']
        if new_city.lower() == city.lower():
            list_of_dicts.append(city_bike_dict)

    return list_of_dicts

def get_stations_info(city):
    station_dict = get_city_data(city)
    if not station_dict:
        print("Error: No bike company found for {}".format(city))
        return None

    network_address = station_dict[0]['href']
    url = "http://api.citybik.es/{}".format(network_address)
    return requests.get(url).json()['network']['stations']

In [25]:
# Confirm city information with the function get_city_data(city)
city = "Lille"
city_info = get_city_data(city)
get_city_data("Lille")

[{'company': ['Keolis'],
  'href': '/v2/networks/vlille',
  'id': 'vlille',
  'location': {'city': 'Lille',
   'country': 'FR',
   'latitude': 50.62925,
   'longitude': 3.057256},
  'name': "V'Lille",
  'source': 'https://opendata.lillemetropole.fr/explore/dataset/vlille-realtime/'}]

In [26]:
# Confirm station information with the function get_stations_info(city)
city = "Lille"
station_info = get_stations_info(city)
#pd.DataFrame(station_info)
city_station = pd.DataFrame(station_info)
city_station

,empty_slots,extra,free_bikes,id,latitude,longitude,name,timestamp
0,18,"{'address': 'Parvis Rotterdam', 'city': 'LILLE...",2,3d91e5c7edcdf745a745ae95b3cf3721,50.637709,3.077478,PARVIS ROTTERDAM,2023-09-25T20:33:27.335000Z
1,22,"{'address': 'ALLEE VICTOR BASCH', 'city': 'LA ...",5,93051bcb05162e8baae7124c1385f2b8,50.645430,3.075116,ROMARIN,2023-09-25T20:33:27.309000Z
2,0,"{'address': 'RUE CHAMBORD', 'city': 'VILLENEUV...",0,fed1d53e44d47f710bad9fe41cea5f22,50.629720,3.133873,RUE CHAMBORD,2023-09-25T20:33:27.101000Z
3,9,"{'address': '205-189 rue de l'Epeule', 'city':...",3,e04498b5eba46941af76fc7285e66325,50.687500,3.158559,MAIRIE DE QUARTIER OUEST,2023-09-25T20:33:27.376000Z
4,4,"{'address': '2 RUE MARCEL HÉNAUX', 'city': 'LI...",12,3212aca4329dc3ebeb67701915f3d6bd,50.614052,3.062195,FAUBOURG D'ARRAS,2023-09-25T20:33:27.327000Z
...,...,...,...,...,...,...,...,...
284,3,"{'address': '15 PLACE PHILLIPPE LEBON', 'city'...",15,a6d8f19f381f449646027e99058503cf,50.627890,3.062090,PLACE PHILIPPE LEBON,2023-09-25T20:33:27.646000Z
285,0,"{'address': '69 AVENUE DE VERDUN', 'city': 'MO...",0,ccdf0e8b02464cddfa24ad550d737733,50.638126,3.102310,VERDUN,2023-09-25T20:33:27.634000Z
286,0,"{'address': '56, RUE PAUL VAILLANT', 'city': '...",0,14fc6d2421bfef3d1b146d769d8a56d5,50.641834,3.022364,COLLEGE LAVOISIER,2023-09-25T20:33:27.634000Z
287,14,"{'address': 'RUE CORNEILLE', 'city': 'VILLENEU...",0,0e89b99fa1d1765167dc7e062e8aa90f,50.623795,3.144815,RUE CORNEILLE,2023-09-25T20:33:27.635000Z


In [27]:
city_station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  289 non-null    int64  
 1   extra        289 non-null    object 
 2   free_bikes   289 non-null    int64  
 3   id           289 non-null    object 
 4   latitude     289 non-null    float64
 5   longitude    289 non-null    float64
 6   name         289 non-null    object 
 7   timestamp    289 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 18.2+ KB


####  For each bike station, use the API to call the latitude, longitude and number of bikes.
##### (Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes).)

In [28]:
import pandas as pd
import requests


'''

    Each function has a specific purpose, such as fetching networks, 
    filtering them by city, retrieving station data, and creating the station data list.
    Also parse the JSON object into a Pandas dataframe.  
    
'''
# Function to retrieve station name, longitude and latitude and total bikes
def get_bike_stations_total(city_name):
    networks = get_networks()
    filtered_networks = filter_networks_by_city(networks, city_name)

    if not filtered_networks:
        print(f"No networks found for {city_name}.")
        return pd.DataFrame()

    chosen_network = filtered_networks[0]
    bike_stations = get_station_data(chosen_network)

    station_data_list = create_station_data_list(bike_stations)

    df = pd.DataFrame(station_data_list)

    print(f"Bike Stations in {city_name} (DataFrame):\n")
    print(df)

    return df

# Function to fetch networks
def get_networks():
    response = requests.get("http://api.citybik.es/v2/networks")
    return response.json()["networks"]


# Function to filter network by city
def filter_networks_by_city(networks, city_name):
    return [network for network in networks if network["location"]["city"].lower() == city_name.lower()]

#Function to retrieve station data
def get_station_data(network):
    network_id = network["id"]
    network_url = f"http://api.citybik.es/v2/networks/{network_id}"
    response = requests.get(network_url)
    return response.json().get("network", {}).get("stations", [])

# Function to create station data list
def create_station_data_list(bike_stations):
    station_data_list = []

    for station in bike_stations:
        latitude = station["latitude"]
        longitude = station["longitude"]
        station_name = station["name"]

        if 'extra' in station and 'ebikes' in station['extra']:
            total_bikes = station["free_bikes"] + station["empty_slots"] + station['extra']['ebikes']
        else:
            total_bikes = station["free_bikes"] + station["empty_slots"]

        station_data = {
            "Station Name": station_name,
            "Latitude": latitude,
            "Longitude": longitude,
            "Total Bikes": total_bikes
        }

        station_data_list.append(station_data)

    return station_data_list

city_name = input("Enter the name of the city: ")
city_bike_total = get_bike_stations_total(city_name)


Bike Stations in Lille (DataFrame):

                 Station Name   Latitude  Longitude  Total Bikes
0            PARVIS ROTTERDAM  50.637709   3.077478           20
1                     ROMARIN  50.645430   3.075116           27
2                RUE CHAMBORD  50.629720   3.133873            0
3    MAIRIE DE QUARTIER OUEST  50.687500   3.158559           12
4            FAUBOURG D'ARRAS  50.614052   3.062195           16
..                        ...        ...        ...          ...
284      PLACE PHILIPPE LEBON  50.627890   3.062090           18
285                    VERDUN  50.638126   3.102310            0
286         COLLEGE LAVOISIER  50.641834   3.022364            0
287             RUE CORNEILLE  50.623795   3.144815           14
288               CHATELLENIE  50.635990   3.135660           20

[289 rows x 4 columns]


In [29]:
city_bike_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station Name  289 non-null    object 
 1   Latitude      289 non-null    float64
 2   Longitude     289 non-null    float64
 3   Total Bikes   289 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 9.2+ KB


##### Put your parsed results into a DataFrame.

In [30]:
# The function created above includes parse the JSON object into Pandas DataFrame
print(type(city_bike_total))

<class 'pandas.core.frame.DataFrame'>


In [32]:
# saving a copy of the dataframe
city_bike_total.to_csv(r'C:\Code\Project_Stat_Model_DataFrame_Store\city_bike_df.csv', index=False)

In [33]:
df_new = pd.read_csv(r'C:\Code\Project_Stat_Model_DataFrame_Store\city_bike_df.csv')
df_new

,Station Name,Latitude,Longitude,Total Bikes
0,PARVIS ROTTERDAM,50.637709,3.077478,20
1,ROMARIN,50.645430,3.075116,27
2,RUE CHAMBORD,50.629720,3.133873,0
3,MAIRIE DE QUARTIER OUEST,50.687500,3.158559,12
4,FAUBOURG D'ARRAS,50.614052,3.062195,16
...,...,...,...,...
284,PLACE PHILIPPE LEBON,50.627890,3.062090,18
285,VERDUN,50.638126,3.102310,0
286,COLLEGE LAVOISIER,50.641834,3.022364,0
287,RUE CORNEILLE,50.623795,3.144815,14


In [34]:
city_bike_total = df_new
city_bike_total

,Station Name,Latitude,Longitude,Total Bikes
0,PARVIS ROTTERDAM,50.637709,3.077478,20
1,ROMARIN,50.645430,3.075116,27
2,RUE CHAMBORD,50.629720,3.133873,0
3,MAIRIE DE QUARTIER OUEST,50.687500,3.158559,12
4,FAUBOURG D'ARRAS,50.614052,3.062195,16
...,...,...,...,...
284,PLACE PHILIPPE LEBON,50.627890,3.062090,18
285,VERDUN,50.638126,3.102310,0
286,COLLEGE LAVOISIER,50.641834,3.022364,0
287,RUE CORNEILLE,50.623795,3.144815,14


In [ ]:
del df_new